# Домашнее задание 2. Сравнение систем автоматического распознавания речи.

В этом задании вам нужно будет сравнить автоматические субтитры YouTube с другими системами.
Можно использовать любой способ скачивания данных, который вам удобен.

Скорее всего, вам будет удобнее работать в Google Colab.

### Формат сдачи задания:

Jupyter-notebook на гитхабе.

### Дедлайн: 

### Описание задания:

#### Загрузка видео

1. Выберите любое видео на YouTube, где есть речь (на любом известном вам языке) 
и автоматические субтитры - при выборе языка субтитров будет написано "создано автоматически".
Пример видео с автоматическими субтитрами https://www.youtube.com/watch?v=f3mhds1z3rE
<br/>Скачайте звук видео и субтитры любым удобным способом. Рекомендованный способ - [youtube-dl](https://github.com/ytdl-org/youtube-dl).
Вы также можете воспользоваться расширениями для браузера и т.п. - напишите в тетрадке, как скачаны файлы.
Сами файлы добавьте в ваш репозиторий с домашкой. - максимум 2 балла при использовании youtube-dl или другой библиотеки

У меня нет сил искать что-то интересное, поэтому я взял ещё одно видео с котами... Ну оно неплохое, и на французском. А то на английском ютуб вообще не ошибается, так хоть немного интересно

Я возьму `pytube`, потому что он ставится через пип, у меня нет прав, чтобы делать через curl. Звук можно достать через библиотеку, а текст через bs4, если парсить xml. Можно и json конечно, но он сложнее. Всё закинул в отедльные файлики, но там в любом случае быстро

In [2]:
!pip install -q pytube

In [72]:
from pytube import YouTube
from bs4 import BeautifulSoup
import pandas as pd

yt = YouTube('https://www.youtube.com/watch?v=Z0QvSQ2sgp4')

sound = yt.streams.filter(only_audio=True).all()[0]
sound.download(filename="sound.mp4")

captions = yt.captions["a.fr"]
soup = BeautifulSoup(captions.xml_captions, 'lxml')
youtube_audio = [p.text for p in soup.find_all("p") if p.text != "\n"]
youtube_audio = " ".join(youtube_audio)
pd.to_pickle(youtube_audio, "youtube_audio.pkl")

/tmp/ipykernel_227/2366014069.py:7: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  sound = yt.streams.filter(only_audio=True).all()[0]
/home/user/conda/lib/python3.9/site-packages/bs4/builder/__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


In [4]:
import subprocess

command = "ffmpeg -i sound.mp4 -ab 160k -ac 2 -ar 44100 -vn sound.wav"
subprocess.call(command, shell=True)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

1

#### Прогон ASR

2. Выберите 3 системы ASR и примените их к вашему файлу со звуком. Используйте разные библиотеки -
те, которые упоминались в семинаре, или любые другие. За использование 2 разных библиотек - 4 балла.
Если вы прогоняете через 3 системы с использованием одной библиотеки (например, SpeechRecognition) - это 2 балла.

Я если честно не знаю, что считать системой ASR, время как обычно 21:55, но в тетрадке к занятию полно разных методов это здесь, в том числе с разными библиотеками, так что поехали

##### 1. `Whisper`

Это делается довольно просто, там уже есть свой класс и мультиязычная моделька

In [5]:
!pip install -q SpeechRecognition pocketsphinx
!pip install -q git+https://github.com/openai/whisper.git soundfile

In [6]:
import whisper

model = whisper.load_model("base")
result = model.transcribe("sound.mp4")
whisper_audio = result["text"]

In [82]:
whisper_audio[:100]

" On dit toujours que les chats sont indépendants, autonomes, voire qu'ils en ont que pour leur gueul"

##### 2. `Vosk`

Тут уже не так просто, потому что нормально он работает только из терминала, но тоже ничего сложного

In [ ]:
!wget https://alphacephei.com/vosk/models/vosk-model-small-fr-0.22.zip
!unzip vosk-model-small-fr-0.22.zip.1
!mv vosk-model-small-fr-0.22 model
!pip install -q vosk

Я не знаю, как тут убрать логи, поэтому, если запустить, то будет много мусора в аутпуте. Не понимаю, что им мешало написать нормальный хелп

In [38]:
!vosk-transcriber -i sound.mp4 -o vosk_out.txt --lang fr

usage: vosk-transcriber [-h] [--model MODEL] [--server] [--list-models]
                        [--list-languages] [--model-name MODEL_NAME]
                        [--lang LANG] [--input INPUT] [--output OUTPUT]
                        [--output-type OUTPUT_TYPE] [--tasks TASKS]
                        [--log-level LOG_LEVEL]
vosk-transcriber: error: unrecognized arguments: --log_level -1


In [44]:
with open("vosk_out.txt", "r") as f:
    vosk_audio = " ".join(x.rstrip("\n") for x in f.readlines()) 

In [46]:
vosk_audio[:50]

'je dis toujours que les chats sont indépendants au'

##### 3. `Wav2vec`

Здесь нужно было покопаться, чтобы найти нужную модельку, но в тетрадке это тоже было

In [ ]:
!pip install -q transformers datasets evaluate jiwer torchaudio huggingsound

In [ ]:
from huggingsound import SpeechRecognitionModel

model = SpeechRecognitionModel(
    "jonatasgrosman/wav2vec2-large-fr-voxpopuli-french", device="cuda"
)
audio_paths = ["sound.mp4"]

wav2vec_audio = model.transcribe(audio_paths)[0]["transcription"]

In [55]:
wav2vec_audio[:50]

'ON DIT TOUJOURS QUE LES CHASSONT INDÉPENDANT AUTON'

#### Оценка качества

3. Оцените качество работ ваших систем и автогенерации YouTube. Для этого вам нужен эталон.
Вы можете отредактировать один из результатов ASR и использовать его в качестве эталона.
Затем посчитайте WER для каждой системы. - 2 балла

Дополнительные баллы выше 8 можно получить за использование библиотек ASR, 
которые не были показаны на семинаре, и другие возможные улучшения.

Ютубовские субтитры на самом деле очень хороши, я это уже давно заметил. Так что адаптировать я буду именно их. Я сам не до конца понимаю, что там говорится, очень уж странные акценты, но так даже интереснее

In [19]:
srt = [
 'on dit toujours que les chats sont',
 "indépendants autonomes voire qu'ils en",
 "ont que pour leur gueule mais c'est pas",
 'vrai oh merde pardon là par exemple tu',
 "aides à faire la vaisselle n'est-ce pas",
 'ouais ouais je cherchais le produit',
 'vaisselle alors imaginons un monde où',
 'les chats seraient serviables merci',
 "c'est bon à ton service bah moi aussi je",
 'veux bien aider à faire la vaisselle non',
 'tu enlèves tes pattes pourquoi tu sais',
 'très bien pourquoi moi je peux aider',
 "alors s'il te plaît il faudrait ranger",
 'tous ces bonbons par couleur',
 'ouais non ça me soûle attends moi je',
 "vais t'aider à bien ranger ta chambre ça",
 "t'as plus besoin t'as déjà lu ça bah",
 "de toute façon on s'en fout il y a",
 "l'animé en vrai c'est pas compliqué hein",
 "ça c'est moche ça c'est de la merde ça",
 "ça pue et ça bah c'est pas si mal en",
 "fait non je déconne c'est tout pourri",
 "mais moi je demande que ça d'aider à",
 'côté ça ah oui oh puis ça me soûle',
 "tiens c'est peut-être tout simplement ça",
 "si les chats sont pas serviables c'est",
 "parce qu'ils voient pas les choses de la",
 'même manière que nous putain truc de',
 "l'enfer ça m'a fait flipper alors",
 "non merci c'est pas un truc de l'enfer",
 "c'est un cintre ça va si je te le pose",
 "là ton truc de l'enfer donc vous voyez",
 "s'il te plaît que les chats peuvent être",
 "serviables stop merci évidemment qu'on peut être serviable",
 "à votre avis qui c'est qui vousffit les",
 'souris pour protéger vos récoltes et toi',
 "tu fais quoi Ba moi j'ai tout bien plié",
 'le linge alors maintenant tu bien mérité',
 "et de fait dodo attends je vais t'aider",
 "non mais là tu m'aides pas juste tu veux",
 "jouer remarquez c'est peut-être ça le",
 'secret ça sert à rien de demander donne',
 'faut que ça vienne de je veux aider',
 'donne mais donne ou pas mais si tu veux',
 'vraiment aider tu peux donner un coup de',
 'pouce'
]
true_audio = " ".join(x for x in srt)

In [2]:
import pandas as pd

whisper_audio = pd.read_pickle("whisper_audio.pkl")
vosk_audio = pd.read_pickle("vosk_audio.pkl")
wav2vec_audio = pd.read_pickle("wav2vec_audio.pkl")
youtube_audio = pd.read_pickle("youtube_audio.pkl")
true_audio = pd.read_pickle("true_audio.pkl")

In [49]:
from evaluate import load
wer = load("wer")

Будет большая проблема в том, чтобы все эти токены сопоставить, их там порядка 2000. Я возьму только срез, иначе это будет просто долго

In [114]:
def evaluate(n_slice):
    references = true_audio.lower().split()[:n_slice]
    for name, predictions in [
        ("whisper", whisper_audio),
        ("vosk", vosk_audio),
        ("wav2vec", wav2vec_audio),
        ("youtube", youtube_audio)
    ]:
        audio_slice = predictions.lower().split()[:n_slice]
        wer_score = wer.compute(predictions=audio_slice, references=references)
        print(name, wer_score)

In [116]:
evaluate(100)

whisper 0.72
vosk 0.92
wav2vec 0.95
youtube 0.02


Ну вот и получается, что ютуб расшифровывает вообще прекрасно, а все остальные не очень. Почему - загадка, но смотри видосики на французском там очень приятно, потому что сами французы субтитры делать не любят. Боже благослови нейросети. Но whisper на самом деле тоже очень хорош, просто в какой-то момент он ошибся в одном слове и всё поехало, если взять меньший сплит, то будет видно. Я не знаю, если честно, как это быстро пофиксить, токенизация тут не поможет, а иначе мне ошибку выдаёт. Не успеваю ничего придумать ((

In [119]:
evaluate(20)

whisper 0.15
vosk 0.65
wav2vec 0.75
youtube 0.0


На срезе поменьше видно, что whisper как минимум не так уж плох, а вообще он довольно прикольен тем, что умеет знаки пунктуации вычленять

Насчёт улучшений не знаю, можно попробовать денойзинг сделать, хотя моделька вроде должна уметь сама, но посмотрим

In [ ]:
!pip install -q speechbrain

In [ ]:
from speechbrain.pretrained import SepformerSeparation as separator
import torchaudio

model = separator.from_hparams(
    source="speechbrain/sepformer-whamr-enhancement",
    savedir='pretrained_models/sepformer-whamr-enhancement',
)

# for custom file, change path
est_sources = model.separate_file(
    path='sound.mp4', 
) 

torchaudio.save("denoised_audio.wav", est_sources[:, :, 0].detach().cpu(), 8000)

torchvision is not available - cannot save figures


Resampling the audio from 44100 Hz to 8000 Hz


In [ ]:
asd

Ну, предположить, что модель будет весить больше 40Гб, я не мог, поэтому сегодня без денойзинга, на A100 не влезает

У меня не работают другие библиотеки, я пытался. Я и так использовал vosk и whisper во втором пункте, но я ничего ещё не могу найти, не знаю, в общем. SpeechRecognition есть конечно, но он был на семинаре, плюс он не интересный. К тому же это должно быть что-то для французского. Есть вот это - https://github.com/maelfabien/french_asr_app/blob/main/asr.py - там опять сетка, для azure нужен ключ, для гугла нужен ключ. В общем печально это всё, зафайнтюнить тоже не успею, да и толку нет, потому что не на чем. Ну ладно, всё равно 10 бы ничего не дало

In [2]:
!pip install -q asrecognition librosa

In [ ]:
from asrecognition import ASREngine

# 1 - Load the ASR engine for a given language (on the first run it may take a while)
asr = ASREngine("fr")

# 2 - Use the loaded ASR engine to transcribe a list of audio files
audio_paths = ["audio.mp4"]
transcriptions = asr.transcribe(audio_paths)

# 3 - Voilà!
print(transcriptions)